In [ ]:
# imports
# !pip install langchain-chroma

In [19]:

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain_chroma import Chroma
from openai import OpenAI
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import numpy as np
# import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import DirectoryLoader, TextLoader

In [20]:
# Load environment variables in a file called .env
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()
db_name = "metaData"

In [25]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("train/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents2 = []
for folder in folders:
    doc_type = os.path.basename(folder)
    if os.path.isdir(folder):
        loader = DirectoryLoader(folder, glob="**/*", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
        folder_docs = loader.load()
    else:
        loader = TextLoader(folder, **text_loader_kwargs)
        folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents2.append(doc)

In [26]:
documents2

[Document(metadata={'source': 'train/DataSet.txt', 'doc_type': 'DataSet.txt'}, page_content="Below are the summary for AppBrewery Virtual Machine's (VM) data\n    \n    Table_name -> AppBrewery\n\n    Attribute_list -> \n        1. VMNAME : Name of the virtual machine.\n        2. State : Indicates whether the VM is ON or OFF.\n        3. Status : Describes the VM's condition (Normal or Special).\n        4. Host : The physical server hosting the VM.\n        5. Cluster : The group of hosts managing the VM.\n        6. Provisioned_Space : Total storage allocated to the VM.\n        7. Used_Space : Storage currently used by the VM.\n        8. HostCPU : CPU allocation on the host machine.\n        9. HostMem : Memory allocated to the VM.\n        10. KN_SNC_DT : Known since date of the VM record.\n        11. Decom_date : Planned decommissioning date of the VM.\n\n    Self table RelationShips : \n        Host Can have multiple VMNAME (VM)\n        Cluster can have multiple Host\n    \n 

In [40]:
# import os
# import glob
# from dotenv import load_dotenv

# # Load environment variables in a file called .env
# load_dotenv()
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

# # Read in documents from all the sub-folders of our knowledgebase
# folders = glob.glob("train/*")

# documents = []
# for folder in folders:
#     doc_type = os.path.basename(folder)
#     if os.path.isdir(folder):
#         for file_path in glob.glob(os.path.join(folder, "**/*"), recursive=True):
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 content = file.read()
#                 documents.append({
#                     'page_content': content,
#                     'metadata': {
#                         'source': doc_type,
#                         'topic': 'machine learning',  # Example topic, you can modify as needed
#                         'author': 'John Doe'  # Example author, you can modify as needed
#                     }
#                 })
#     else:
#         with open(folder, 'r', encoding='utf-8') as file:
#             content = file.read()
#             documents.append({
#                 'page_content': content,
#                 'metadata': {
#                     'source': doc_type,
#                     'topic': 'machine learning',  # Example topic, you can modify as needed
#                     'author': 'John Doe'  # Example author, you can modify as needed
#                 }
#             })

# # Print the documents in JSON format
# import json
# for doc in documents:
#     print(json.dumps(doc, indent=4))

In [38]:
documents

[{'page_content': "Below are the summary for AppBrewery Virtual Machine's (VM) data\n    \n    Table_name -> AppBrewery\n\n    Attribute_list -> \n        1. VMNAME : Name of the virtual machine.\n        2. State : Indicates whether the VM is ON or OFF.\n        3. Status : Describes the VM's condition (Normal or Special).\n        4. Host : The physical server hosting the VM.\n        5. Cluster : The group of hosts managing the VM.\n        6. Provisioned_Space : Total storage allocated to the VM.\n        7. Used_Space : Storage currently used by the VM.\n        8. HostCPU : CPU allocation on the host machine.\n        9. HostMem : Memory allocated to the VM.\n        10. KN_SNC_DT : Known since date of the VM record.\n        11. Decom_date : Planned decommissioning date of the VM.\n\n    Self table RelationShips : \n        Host Can have multiple VMNAME (VM)\n        Cluster can have multiple Host\n    \n    Foreign Table relationShips :\n         None",
  'metadata': {'source':

In [34]:


# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()


In [42]:

# Create vectorstore

vectorstore = Chroma.from_documents(documents=documents2, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

OperationalError: attempt to write a readonly database

In [9]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)


In [10]:
retriever = vectorstore.as_retriever()

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [12]:
system_prompt=(""" You are expert in writing sql statements about AppBrewery's Database
               Return responses in sql statements when necessary
    {context}""")
    
    
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    ])


In [13]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [14]:
answer= rag_chain.invoke({"input": "How many Virtual Machines are there in AppBrewery?"})
answer['answer']

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'```sql\nSELECT COUNT(*) AS Total_Virtual_Machines\nFROM AppBrewery_VM;\n```'

In [27]:
answer= rag_chain.invoke({"input": "Do you know anything about AppBrewery?"})
answer['answer']

"Yes, I can provide information and assist you with SQL statements related to AppBrewery's Database. If you have specific questions or need assistance with SQL queries, feel free to ask!"

In [26]:
answer= rag_chain.invoke({"input": "How many VMs are there in AppBrewery ?"})
answer['answer']

'There are a total of 4 VMs in AppBrewery.'

In [15]:
def chat(question, history):
    result = rag_chain.invoke({"input": question})
    return result["answer"]

In [16]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
